In [ ]:
!pip install -qU pip
!pip install -q rasterio cartopy dataretrieval contextily pypandoc
!pip install -qq crewai==0.75.0 crewai-tools==0.13.2 "tokenizers==0.22.0" requests==2.32.4

In [ ]:
!apt-get -qq update
!DEBIAN_FRONTEND=noninteractive apt-get install -y -qq pandoc texlive-xetex > /dev/null 2>&1

Now we download the backend hydrologic model. This takes about 5 minutes to set it up. So, be patient.

In [ ]:
!rm -rf AQUAH_v0.1
!git clone https://github.com/chrimerss/AQUAH_v0.1.git
%cd /content/AQUAH_v0.1
# Download EF5 Model for simulation, it takes about 1 or 2 minutes
!rm -rf EF5
!git clone https://github.com/chrimerss/EF5.git
%cd EF5
# Compile EF5 Model
!autoreconf --force --install > /dev/null
!./configure > /dev/null 2>&1
!sed -i 's/-Werror//g' Makefile
!make -s > /dev/null 2>&1
%cd ..
print('Successfully installed EF5.')

Now, you need to configure with your own API key from openAI. Go to https://platform.openai.com/settings/profile/user to look up your API key.

![image.png](https://github.com/chrimerss/AQUAH_v0.1/blob/master/resource/screenshot.jpg?raw=true)

For the "Name" entry, you can put OPENAI and "Value" to be the key.

In [4]:
from google.colab import userdata

api_key = userdata.get('OPENAI')

In [ ]:
from importlib import reload
import tools.aquah_run
reload(tools.aquah_run)
from tools.aquah_run import aquah_run
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["OPENAI_API_KEY"] = api_key
llm_model_name = 'gpt-4o'
# llm_model_name = 'claude-4-sonnet-20250514'
# llm_model_name = 'gemini-2.5-flash-preview-05-20'
# llm_model_name = 'claude-4-opus-20250514'
aquah_run(llm_model_name)